<a href="https://colab.research.google.com/github/hendrikyong/CVNL_Assignment_1/blob/main/CVNL_Assg1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/sonos/nlu-benchmark.git



fatal: destination path 'nlu-benchmark' already exists and is not an empty directory.


In [ ]:
!ls nlu-benchmark/2017-06-custom-intent-engines/AddToPlaylist

Alexa_metrics.json   Snips_metrics_full.json	    train_AddToPlaylist.json
api.ai_metrics.json  Snips_metrics.json		    validate_AddToPlaylist.json
Luis_metrics.json    train_AddToPlaylist_full.json  Wit_metrics.json


In [ ]:
import json
from pathlib import Path

# Example: Path to your training file
train_file = "nlu-benchmark/2017-06-custom-intent-engines/AddToPlaylist/train_AddToPlaylist.json"

# Initialize lists to store texts and intents
texts = []
intents = []

# Parse the JSON file
with open(train_file, 'r') as f:
    data = json.load(f)

# Loop through each entry in the dataset
for entry in data["AddToPlaylist"]:
    # Concatenate text fragments to form a complete sentence
    sentence = ''.join([fragment['text'] for fragment in entry['data']])
    texts.append(sentence)
    intents.append("AddToPlaylist")  # Use the folder name as the intent

# Display the parsed data
print("Texts:", texts[:5])  # Display first 5 examples
print("Intents:", intents[:5])


Texts: ['add Stani, stani Ibar vodo songs in my playlist música libre', 'add this album to my Blues playlist', 'Add the tune to the Rage Radio playlist.', 'Add WC Handy to my Sax and the City playlist', 'Add BSlade to women of k-pop playlist']
Intents: ['AddToPlaylist', 'AddToPlaylist', 'AddToPlaylist', 'AddToPlaylist', 'AddToPlaylist']


In [ ]:
!pip install torchtext


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.0 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install torchtext
from torchtext.vocab import vocab


Looking in indexes: https://download.pytorch.org/whl/cpu


OSError: /usr/local/lib/python3.11/dist-packages/torchtext/lib/libtorchtext.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs

In [ ]:
import os
import json
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from sklearn.model_selection import train_test_split

# Custom Dataset class
class IntentDataset(Dataset):
    def __init__(self, texts, labels, vocab, label_to_idx):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.label_to_idx = label_to_idx

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokens = [self.vocab.get(token, self.vocab["<UNK>"]) for token in text.split()]
        return torch.tensor(tokens, dtype=torch.long), torch.tensor(self.label_to_idx[label], dtype=torch.long)

def load_data(base_dir):
    intents = []
    texts = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.startswith("train_") and file.endswith(".json"):
                intent = os.path.basename(root)  # Intent name is the folder name
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        for entry in data:
                            print(f"Debug entry: {entry}")
                            texts.append(entry if isinstance(entry, str) else '')
                            intents.append(intent)
                except UnicodeDecodeError as e:
                    print(f"UnicodeDecodeError while reading {file_path}: {e}")
                except json.JSONDecodeError as e:
                    print(f"JSONDecodeError while parsing {file_path}: {e}")
    return texts, intents




# Build vocabulary
def build_vocab(texts, min_freq=1):
    counter = Counter(token for text in texts for token in text.split())
    vocab = {word: idx + 2 for idx, (word, freq) in enumerate(counter.items()) if freq >= min_freq}
    vocab["<PAD>"] = 0
    vocab["<UNK>"] = 1
    return vocab

# Data directory
data_dir = "nlu-benchmark/2017-06-custom-intent-engines"

# Load data
texts, intents = load_data(data_dir)

# Build vocabulary and label mappings
vocab = build_vocab(texts)
labels = list(set(intents))
label_to_idx = {label: idx for idx, label in enumerate(labels)}

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, intents, test_size=0.2, random_state=42)

# Create datasets and data loaders
train_dataset = IntentDataset(train_texts, train_labels, vocab, label_to_idx)
val_dataset = IntentDataset(val_texts, val_labels, vocab, label_to_idx)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: pad_batch(x))
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: pad_batch(x))

# Collate function
def pad_batch(batch):
    texts, labels = zip(*batch)
    texts = pad_sequence(texts, batch_first=True, padding_value=vocab["<PAD>"])
    labels = torch.tensor(labels, dtype=torch.long)
    return texts, labels

# Define the model
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super(RNNClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.rnn = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.rnn(x)
        out = self.fc(hidden.squeeze(0))
        return out

# Initialize the model
model = RNNClassifier(len(vocab), embed_size=100, hidden_size=128, output_size=len(labels))

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        for texts, labels in train_loader:
            outputs = model(texts)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Evaluate on validation set
        model.eval()
        total, correct = 0, 0
        with torch.no_grad():
            for texts, labels in val_loader:
                outputs = model(texts)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f"Epoch {epoch + 1}/{epochs}, Validation Accuracy: {correct / total:.4f}")

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer)


Debug entry: BookRestaurant
Debug entry: BookRestaurant
Debug entry: SearchScreeningEvent
Debug entry: SearchScreeningEvent
Debug entry: PlayMusic
UnicodeDecodeError while reading nlu-benchmark/2017-06-custom-intent-engines/PlayMusic/train_PlayMusic_full.json: 'utf-8' codec can't decode byte 0xed in position 150732: invalid continuation byte
Debug entry: SearchCreativeWork
Debug entry: SearchCreativeWork
Debug entry: GetWeather
Debug entry: GetWeather
Debug entry: RateBook
Debug entry: RateBook
Debug entry: AddToPlaylist
Debug entry: AddToPlaylist
Epoch 1/10, Validation Accuracy: 0.3333
Epoch 2/10, Validation Accuracy: 0.6667
Epoch 3/10, Validation Accuracy: 0.6667
Epoch 4/10, Validation Accuracy: 1.0000
Epoch 5/10, Validation Accuracy: 1.0000
Epoch 6/10, Validation Accuracy: 1.0000
Epoch 7/10, Validation Accuracy: 1.0000
Epoch 8/10, Validation Accuracy: 1.0000
Epoch 9/10, Validation Accuracy: 1.0000
Epoch 10/10, Validation Accuracy: 1.0000
